In [1]:
import os.path
import re
from gensim import corpora
from gensim.models import LsiModel
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt
import nltk
from gensim.models import Phrases
from gensim import corpora
from gensim.models import LsiModel, LdaModel
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from rdflib import Graph
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import precision_score, recall_score
import os
import pickle
import torch
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoModelForMaskedLM, AutoTokenizer
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ebrahim/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
def load_data(path, file_name):
    """
    Input  : path and file_name
    Purpose: loading text file
    Output : list of paragraphs/documents, titles,
             and subject lists
    """
    titles = []
    subjects_list = []

    with open("output_2.txt", "r", encoding="utf-8") as file:
        lines = file.readlines()

    # Extract titles from the content
    collecting_title = False
    current_title = ""

    for line in lines:
        if line.startswith("Title:"):
            # If we were previously collecting a title, we append it to the titles list
            if collecting_title:
                titles.append(current_title.strip())
                current_title = ""
            collecting_title = True
            current_title += line.replace("Title:", "").strip()
        elif collecting_title:
            current_title += " " + line.strip()

    # Append the last title if any
    if current_title:
        titles.append(current_title.strip())
    title_list = []
    abstract_list = []
    subject_list = []

    for title in titles:
        # Splitting on "Abstract:" and taking the first part as title and the second part as abstract
        parts = title.split("Abstract:")
        
        title_list.append(parts[0].strip())
        abstract_list.append(parts[1].split(" Subject:")[0].strip())
        subject_list.append(parts[1].split(" Subject:")[1].strip())

    print("Total Number of Documents:", len(abstract_list))

    return abstract_list, title_list, subject_list

In [3]:
def preprocess_data(doc_set, ngram_range=(1, 3)):
    """
    Input  : document list
    Purpose: preprocess text (tokenize, removing stopwords, and stemming)
    Output : preprocessed text
    """
    # Initialize regex tokenizer
    tokenizer = RegexpTokenizer(r'\w+')
    # Create English stop words list
    en_stop = set(stopwords.words('english'))
    # Create p_stemmer of class PorterStemmer
    p_stemmer = PorterStemmer()
    # List for tokenized documents in loop
    texts = []
    # Loop through the document list
    for i in doc_set:
        # Clean and tokenize the document string
        raw = i.lower()
        raw = re.sub(r'(\+)?\d+[ ]?\d*[ ]?\d*[ ]?\d*', ' ', raw)
        # removing characters like ? ! : ; \n \
        raw = re.sub('[*.?!,:;/&]', ' ', raw)
        # removing words with fewer than 3 characters
        raw = re.sub(r'\b\w{1,2}\b', '', raw)
        tokens = tokenizer.tokenize(raw)
        # removing stop words from tokens
        stopped_tokens = [i for i in tokens if not i in en_stop]
        
        # Generate n-grams within the specified range
        ngram_tokens = []
        for n in range(ngram_range[0], ngram_range[1] + 1):
            ngrams = list(nltk.ngrams(stopped_tokens, n))
            ngram_tokens.extend(['_'.join(gram) for gram in ngrams])
        
        # Stem tokens (You can uncomment this line to perform stemming)
        # stemmed_tokens = [p_stemmer.stem(i) for i in ngram_tokens]
        stemmed_tokens = ngram_tokens
        # Add tokens to the list
        texts.append(stemmed_tokens)
    return texts

def prepare_corpus(doc_clean, include_bigrams=True):
    """
    Input  : clean document
    Purpose: create a term dictionary of our corpus and convert a list of documents (corpus) into a Document Term Matrix
    Output : term dictionary and Document Term Matrix
    """
    # Optionally, create bigrams if include_bigrams is True
    if include_bigrams:
        bigram = Phrases(doc_clean)
        doc_clean = list(bigram[doc_clean])

    # Creating the term dictionary of our corpus, where every unique term is assigned an index.
    dictionary = corpora.Dictionary(doc_clean)

    # Converting the list of documents (corpus) into a Document Term Matrix using the dictionary prepared above.
    doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

    return dictionary, doc_term_matrix

from gensim.models.lsimodel import stochastic_svd

def create_gensim_lsa_model(doc_clean, number_of_topics, words, include_bigrams=True):
    """
    Input  : clean document, number of topics, and number of words associated with each topic
    Purpose: create LSA model using gensim
    Output : return LSA model
    """
    # Preprocess the data
    #preprocessed_text = preprocess_data(doc_clean)

    # Generate LSA model with or without bigrams
    dictionary, doc_term_matrix = prepare_corpus(doc_clean, include_bigrams=include_bigrams)
    
    # Initialize LSI model
    lsamodel = LsiModel(doc_term_matrix, num_topics=number_of_topics, id2word=dictionary, chunksize= 1000)  # train model

    return lsamodel, dictionary, doc_term_matrix

In [4]:
document_list,titles,subjects =load_data("","")

Total Number of Documents: 64361


In [5]:
document_list,titles=document_list,titles
clean_text=preprocess_data(document_list)

In [6]:
import ast
from collections import Counter

subject_lists = [ast.literal_eval(subject_str) for subject_str in subjects]

# Flatten the list of lists to get a single list of all topics
all_topics = [topic for sublist in subject_lists for topic in sublist]

# Count the occurrences of each topic
topic_counts = Counter(all_topics)

# Print statistics about the number of papers for each topic
print("Statistics about the number of papers for each topic:")
for topic, count in topic_counts.items():
    print(f"- {topic}: {count} papers")

def papers_with_subject(abstracts, titles, subjects, target_subject):
    """
    Return papers that have a specific subject in their subject list.
    """
    papers_with_target_subject_abstracts = []
    papers_with_target_subject_titles = []

    for abstract, title, subject_list in zip(abstracts, titles, subjects):
        if target_subject in subject_list:
            papers_with_target_subject_abstracts.append(abstract)
            papers_with_target_subject_titles.append(title)

    return papers_with_target_subject_abstracts, papers_with_target_subject_titles

target_subject = "Metals and Alloys"
target_subject_abstracts, target_subject_titles = papers_with_subject(document_list, titles, subjects, target_subject)

Statistics about the number of papers for each topic:
- Condensed Matter Physics: 20571 papers
- Electronic, Optical and Magnetic Materials: 7371 papers
- Civil and Structural Engineering: 2011 papers
- Ceramics and Composites: 3717 papers
- General Materials Science: 17821 papers
- Mechanics of Materials: 13175 papers
- Metals and Alloys: 8432 papers
- Mechanical Engineering: 11449 papers
- Geology: 147 papers
- Geotechnical Engineering and Engineering Geology: 344 papers
- Multidisciplinary: 11 papers
- Materials Chemistry: 17200 papers
- Surfaces, Coatings and Films: 6907 papers
- Surfaces and Interfaces: 5698 papers
- Earth-Surface Processes: 46 papers
- Materials Science (miscellaneous): 62 papers
- Instrumentation: 3327 papers
- Nuclear and High Energy Physics: 3791 papers
- General Chemistry: 8765 papers
- Electrical and Electronic Engineering: 2301 papers
- General Chemical Engineering: 2772 papers
- General Engineering: 3064 papers
- Inorganic Chemistry: 2391 papers
- Organic 

In [7]:
from collections import Counter
import string

# Download stop words set
nltk.download('stopwords')
STOPWORDS = set(stopwords.words('english'))

def tokenize(text):
    # Simple function to tokenize text
    tokens = text.split()
    # Remove punctuation, convert to lowercase, and filter out single characters and numbers
    return [token.strip(string.punctuation).lower() for token in tokens if token and len(token) > 1 and not token.isdigit() and token.lower() not in STOPWORDS]

# Tokenize topics and abstracts
topic_tokens = [tokenize(topic) for topic in titles]
abstract_tokens = [tokenize(abstract) for abstract in document_list]

# Flatten lists and get counts
all_topic_tokens = [token for sublist in topic_tokens for token in sublist]
all_abstract_tokens = [token for sublist in abstract_tokens for token in sublist]


topic_counter = Counter(all_topic_tokens)
abstract_counter = Counter(all_abstract_tokens)

print("Statistics:")
print("Number of topics:", len(titles))
print("Number of abstracts:", len(document_list))
print("Total topic tokens:", len(all_topic_tokens))
print("Total abstract tokens:", len(all_abstract_tokens))
print("Unique topic tokens:", len(topic_counter))
print("Unique abstract tokens:", len(abstract_counter))
print("\nTop 10 topic terms:")
for term, freq in topic_counter.most_common(100):
    print(term, "-", freq)
print("\nTop 10 abstract terms:")
for term, freq in abstract_counter.most_common(100):
    print(term, "-", freq)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ebrahim/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Statistics:
Number of topics: 64361
Number of abstracts: 64361
Total topic tokens: 578133
Total abstract tokens: 5197254
Unique topic tokens: 61358
Unique abstract tokens: 237424

Top 10 topic terms:
temperature - 8208
thermal - 6506
properties - 5458
effect - 4971
transition - 4644
alloys - 4612
study - 4001
x-ray - 3967
cement - 3697
high - 3264
phase - 2674
structure - 2592
effects - 2556
alloy - 2476
diffraction - 2395
concrete - 2370
influence - 2292
analysis - 2261
using - 2243
corrosion - 2225
surface - 2197
mechanical - 2105
low - 1997
wear - 1994
conductivity - 1959
behavior - 1916
steel - 1886
films - 1871
aluminum - 1838
test - 1768
studies - 1761
microstructure - 1642
materials - 1635
composite - 1630
metal - 1617
magnetic - 1611
compression - 1537
behaviour - 1522
synthesis - 1515
dependence - 1510
composites - 1381
electron - 1368
characterization - 1360
carbon - 1327
performance - 1234
formation - 1233
method - 1229
strength - 1211
investigation - 1208
system - 1203
calc

In [8]:
number_of_topics=100
words=3000
model_, dictionary_, doc_term_matrix_ =create_gensim_lsa_model(clean_text, number_of_topics,words)
print('Done')

Done


In [12]:
# Applying WordNet Expansion on words in LSA clusters

unique_words_ = set()
for i, _ in model_.show_topics():
    topic = model_.show_topic(i, topn= 1000)
    for word, score in topic:
        unique_words_.add(word)
               
print(f'number of unique words: {len(unique_words_)}')

number of unique words: 18388


In [8]:
# WordNet Term Expansion
from nltk.corpus import wordnet
from nltk.corpus.reader import Synset
import json
nltk.download('wordnet')
nltk.download('omw-1.4')
def get_parents(obj):
    """
    Get direct hypernyms
    """
    return obj.hypernyms() + obj.instance_hypernyms()
def _get_hierarchies(obj, level=-1):
    
    if level == 0:
        # stop if reached a certain number of levels.
        return [[]]
    
    parents = get_parents(obj)
    if not parents:
        return [[]]

    hierarchies = []
    for parent in parents:
        tmp = _get_hierarchies(parent, level-1)
        for hierarchy in tmp:
            hierarchy.append(parent)
        hierarchies = hierarchies + tmp
    return hierarchies


def get_hierarchies(word, level=-1):
    normalized = word.lower().replace(' ', '_')
    
    # a single word may have multiple synsets
    entries = wordnet.synsets(normalized, pos=wordnet.NOUN)
    filtered_entries = list(filter(lambda x: normalized == x.lemma_names()[0].lower(), entries))

    # only use filtered entries if something is left.
    if filtered_entries:
        entries = filtered_entries

    hierarchies = []
    for entry in entries:
        hierarchies_of_entry = _get_hierarchies(entry, level)
        for hierarchy in hierarchies_of_entry:
            hierarchy.append(entry)
            hierarchies.append(hierarchy)
    return hierarchies

[nltk_data] Downloading package wordnet to /home/ebrahim/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/ebrahim/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [9]:
import re

unique_words_expanded = set()

for word in unique_words_:
    # Check if the word has three or fewer characters
    if len(word) <= 3:
        continue
    
    # Check if the word contains a number using regular expressions
    if re.search(r'\d', word):
        continue
    
    # Get WordNet hypernyms of two levels
    hierarchies = get_hierarchies(word, 1)
    
    if not hierarchies:
        # No expansion, append word as it is
        unique_words_expanded.add(word.lower().replace('_', ' '))
        continue
        
    # For every item in the hierarchy except the original
    for hierarchy in hierarchies[:-1]:
        for item in hierarchy:
            for lemma_name in item.lemma_names():
                word = lemma_name.lower().replace('_', ' ')
                if len(word) <= 3:
                    continue
                
                # Check if the word contains a number using regular expressions
                if re.search(r'\d', word):
                    continue
                unique_words_expanded.add(word)
        break
print(f'Number of unique words after WordNet expansion: {len(unique_words_expanded)}')


NameError: name 'unique_words_' is not defined

In [68]:
unique_words_expanded

{'abstract thought',
 'accumulation',
 'achieved',
 'acier inoxydable',
 'acoustic emission',
 'acquisition',
 'actinic radiation',
 'actinic ray',
 'action',
 'activity',
 'actuation',
 'adhesive primer',
 'adhesive primer plasma',
 'adhesive systems',
 'administrative district',
 'administrative division',
 'aggregation',
 'ainsi que',
 'air abrasion',
 'air dry',
 'air dry nitrogen',
 'alkali content',
 'alkali silica',
 'alkali silica reaction silica reaction asr',
 'alkali silica silica reaction',
 'alkaline media',
 'allowing generation fan',
 'alloy',
 'also',
 'also affected',
 'also observed',
 'alteration',
 'aluminum alloy',
 'ammoniaque',
 'amorphous',
 'amount',
 'amusement',
 'analyzed',
 'analyzed detail',
 'analyzed using',
 'angular pressing',
 'animal material',
 'applied stress',
 'après',
 'arc light curing',
 'arisen',
 'around',
 'around joint',
 'around joint intersection',
 'arrangement',
 'array',
 'artefact',
 'artifact',
 'asr development',
 'asr gel',
 'asr 

In [10]:
# Compare against ontology
import sys
import re
sys.path.append('../')
from evaluation.ontology import sparql, walk
from evaluation import ontology

results =f"""

            SELECT distinct ?id ?label ?comment WHERE {{
                
                OPTIONAL{{ ?id rdfs:label ?label . }}
                #OPTIONAL{{ ?id rdfs:comment ?comment . }}
            }}
        """


namespaces = """
    prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    prefix owl: <http://www.w3.org/2002/07/owl#>
    prefix skos: <http://www.w3.org/2004/02/skos/core#>
    prefix ns2: <http://www.w3.org/2006/time#>
"""
def separate_words(s):
    # Use regex to split camelCase or PascalCase strings
    words = re.sub('([a-z0-9])([A-Z])', r'\1 \2', s)
    return words.lower()

def _owl_get_classes(graph) -> list[tuple[str, str]]:
    results = graph.query(
        f"""
            {namespaces}

            SELECT distinct ?id ?label ?comment ?skoslabel WHERE {{
                {{
                    ?id a owl:Class .
                    
                }} UNION {{
                    ?id a rdfs:Class .
                }} UNION {{
                    ?subclass rdfs:subClassOf ?id .
                }} UNION {{
                    ?id a owl:ObjectProperty .
                }} UNION {{
                    ?id a owl:DatatypeProperty .
                }} UNION {{
                    ?id a owl:NamedIndividual .
                }}
                OPTIONAL {{ 
                        ?id rdfs:label ?label . 
                        FILTER(LANG(?label) = "en")
                    }}
                OPTIONAL {{ 
                        ?id skos:prefLabel ?skoslabel . 
                        FILTER(LANG(?skoslabel) = "en")
                    }}
                FILTER (!isBlank(?id))
            }}
        """
    )
    id_labels = []
    for result in results:
        if result.id and not result.label and not result.skoslabel:
            id_labels.append(( result.id.toPython(), separate_words(result.id.toPython().split('#')[-1] if '#' in result.id.toPython() else result.id.toPython().split('/')[-1])))
        elif result.label:
            #id_labels.append((result.id.toPython(), result.label.value))
            id_labels.append((separate_words(result.id.toPython().split('#')[-1] if '#' in result.id.toPython() else result.id.toPython().split('/')[-1]), ' '.join(re.findall('[A-Z][^A-Z]*', result.label.value)).lower()))
        elif result.skoslabel:
            id_labels.append((separate_words(result.id.toPython().split('#')[-1] if '#' in result.id.toPython() else result.id.toPython().split('/')[-1]), ' '.join(re.findall('[A-Z][^A-Z]*', result.skoslabel.value)).lower()))
        elif result.comment:
            id_labels.append((separate_words(result.id.toPython().split('#')[-1] if '#' in result.id.toPython() else result.id.toPython().split('/')[-1]), result.comment.value.lower()))
    return id_labels

In [87]:
#g = ontology.sparql.graph_from(f'/home/ebrahim/ontology_evaluation/ontologies/PMDCO.ttl')
#g = ontology.sparql.graph_from(f'/home/ebrahim/ontology_evaluation/ontologies/NanoMine.ttl') # NanoMine.ttl # enanomapper.owl # MaterialsMine.ttl
g = ontology.sparql.graph_from(f'/home/ebrahim/ontology_evaluation/ontologies/DISO.ttl') # DISO.ttl # CHAMEO.ttl # CIF-core.ttl
ontology_concepts_list = _owl_get_classes(g)
words=3000
ontology_concepts = set()
for concept, i in ontology_concepts_list:
    ontology_concepts.add(i)

ontology_document = ' '.join([i.replace('_', ' ') for concept, i in ontology_concepts_list])
#print(ontology_document)

clean_text_o = preprocess_data(ontology_concepts)
model_o, dictionary_o, doc_term_matrix_o = create_gensim_lsa_model(clean_text_o, 1, words, include_bigrams=True)

# Extract unique words from the LSA model
ontology_vocabulary = set()
for i, _ in model_o.show_topics():
    topic = model_o.show_topic(i, topn=len(ontology_concepts))
    for word, score in topic:
        ontology_vocabulary.add(word)
print("length", len(ontology_vocabulary))
print(ontology_vocabulary)

length 53
{'coordinate', 'microscopy', 'component_basis', 'z_axis_coordinate', 'second_axis_vector', 'family', 'axis_component', 'first_axis_component', 'burgers_vector', 'x_axis', 'magnitude', 'direction', 'lattice_plane', 'quantity_value', 'third_axis_vector', 'slip_plane_normal', 'third', 'axis_vector', 'first', 'second_axis_component', 'third_axis_component', 'y_axis', 'vector_magnitude', 'third_axis', 'vector', 'x_axis_coordinate', 'crystal', 'family_crystal', 'vector_component', 'first_axis_vector', 'lattice', 'defect', 'second', 'basis', 'vector_origin', 'coordinate_vector', 'position_vector', 'axis', 'y_axis_coordinate', 'quantity_kind', 'crystal_structure', 'vector_component_basis', 'second_axis', 'z_axis', 'slip_plane', 'first_axis', 'line_defect', 'position', 'plane', 'burgers', 'origin', 'kind', 'component'}


In [88]:
set(all_topics)

{'Accounting',
 'Acoustics and Ultrasonics',
 'Aerospace Engineering',
 'Aging',
 'Agricultural and Biological Sciences (miscellaneous)',
 'Agronomy and Crop Science',
 'Analysis',
 'Analytical Chemistry',
 'Anatomy',
 'Anesthesiology and Pain Medicine',
 'Animal Science and Zoology',
 'Anthropology',
 'Applied Mathematics',
 'Applied Microbiology and Biotechnology',
 'Applied Psychology',
 'Aquatic Science',
 'Archeology',
 'Architecture',
 'Artificial Intelligence',
 'Arts and Humanities (miscellaneous)',
 'Astronomy and Astrophysics',
 'Atmospheric Science',
 'Atomic and Molecular Physics, and Optics',
 'Automotive Engineering',
 'Behavioral Neuroscience',
 'Biochemistry',
 'Biochemistry (medical)',
 'Biochemistry, Genetics and Molecular Biology (miscellaneous)',
 'Bioengineering',
 'Biological Psychiatry',
 'Biomaterials',
 'Biomedical Engineering',
 'Biophysics',
 'Biotechnology',
 'Building and Construction',
 'Business and International Management',
 'Cancer Research',
 'Cardiol

In [89]:
# getting the domain papers
from transformers import BertTokenizer, BertModel
import torch

# Load pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'  # Example model, replace with desired model
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

# Function to compute embedding for a word
def get_word_embedding(word):
    inputs = tokenizer(word, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    # Extract embedding for the first token ([CLS] token)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

# Example target subject
target_subject = 'Nanomaterials'#'Characterization'

# Define threshold for confidence score
threshold = 0.80  # Example threshold, adjust as needed

# Compute aggregated embedding for the target subject
target_embedding = np.mean([get_word_embedding(word) for word in target_subject.split()], axis=0)

In [16]:
subject_embeddings = []
for subject in list(set(all_topics)):
    # Example code to compute similarity with each subject's embedding
    subject_embeddings.append(np.mean([get_word_embedding(word) for word in subject.split()], axis=0))

In [18]:
# Compute semantic similarity and confidence score for each subject
mapped_subjects = []
for i, subject_embedding in enumerate(subject_embeddings):
    # Example code to compute similarity with each subject's embedding
    similarity = np.dot(target_embedding, subject_embedding) / (np.linalg.norm(target_embedding) * np.linalg.norm(subject_embedding))
    confidence_score = (similarity + 1) / 2  # Normalize to range [0, 1]
    # Filter out mapped subjects based on threshold
    if confidence_score >= threshold:
        mapped_subjects.append((list(set(all_topics))[i], confidence_score))

# Sort mapped subjects by confidence score (highest to lowest)
mapped_subjects = sorted(mapped_subjects, key=lambda x: x[1], reverse=True)

In [19]:
mapped_subjects

[('Biomaterials', 0.977644145488739),
 ('Electrochemistry', 0.8948090374469757),
 ('Bioengineering', 0.8838181793689728),
 ('General Pharmacology, Toxicology and Pharmaceutics', 0.8743786215782166),
 ('Optometry', 0.8731647729873657),
 ('Biophysics', 0.8728172183036804),
 ('Geochemistry and Petrology', 0.8646673560142517),
 ('Health, Toxicology and Mutagenesis', 0.8638242781162262),
 ('Acoustics and Ultrasonics', 0.8575528860092163),
 ('General Immunology and Microbiology', 0.8571594059467316),
 ('Pharmacology (nursing)', 0.85525843501091),
 ('Microbiology (medical)', 0.854907214641571),
 ('Geotechnical Engineering and Engineering Geology', 0.8539265990257263),
 ('Catalysis', 0.8514692187309265),
 ('Pharmacology (medical)', 0.8497039377689362),
 ('Applied Microbiology and Biotechnology', 0.8483344316482544),
 ('Spectroscopy', 0.8476815223693848),
 ('General Economics, Econometrics and Finance', 0.8465488255023956),
 ('Environmental Chemistry', 0.8461267650127411),
 ('Neurology (clinica

In [23]:
target_subject_abstracts, target_subject_titles = [], []
# Print mapped subjects with confidence scores
for mapped_subject, confidence_score in mapped_subjects[0:1]:
    #print(f"{mapped_subject}: {confidence_score}")
    target_abstracts, target_titles = papers_with_subject(document_list, titles, subjects, mapped_subject)
    target_subject_abstracts.extend(target_abstracts)
    target_subject_titles.extend(target_titles)
    
print('len of papers in this domain', len(target_subject_titles))


len of papers in this domain 908


In [90]:
target_subject_abstracts, target_subject_titles = [], []
target_abstracts, target_titles = papers_with_subject(document_list, titles, subjects, 'General Materials Science') #'General Materials Science' # 'Mechanics of Materials'
target_subject_abstracts.extend(target_abstracts)
target_subject_titles.extend(target_titles)
print('len of papers in this domain', len(target_subject_titles))

len of papers in this domain 17821


In [93]:
clean_text=preprocess_data(target_subject_abstracts)
number_of_topics=20#len(clean_text)
words=3000
model, dictionary, doc_term_matrix =create_gensim_lsa_model(clean_text, number_of_topics,words)
print('Done')

Done


In [94]:
unique_words = set()
for i, _ in model.show_topics():
    topic = model.show_topic(i, topn= 1000)
    for word, score in topic:
        unique_words.add(word)
               
print(f'number of unique words: {len(unique_words)}')

number of unique words: 8259


In [110]:
import re

unique_words_expanded = set()

# add the reference books terms
#with open('Reference Books/callister/1_terms.txt', "r", encoding="utf-8") as file:
with open('Reference Books/Askeland/3_terms.txt', "r", encoding="utf-8") as file:
    # Iterate through each line
    for word in file:
        word = word.strip()
        unique_words.add(word)

with open('Reference Books/callister/3_terms.txt', "r", encoding="utf-8") as file:
    # Iterate through each line
    for word in file:
        word = word.strip()
        unique_words.add(word)
    
for word in unique_words:
    # Check if the word has three or fewer characters
    if len(word) <= 3:
        continue
    
    # Check if the word contains a number using regular expressions
    if re.search(r'\d', word):
        continue
    
    # Get WordNet hypernyms of one level
    hierarchies = get_hierarchies(word.lower().replace('_', ' '), 1)
    
    if not hierarchies:
        # No expansion, append word as it is
        #unique_words_expanded.add(word.lower().replace('_', ' '))
        continue
        
    # For every item in the hierarchy except the original
    for hierarchy in hierarchies[:-1]:
        for item in hierarchy:
            for lemma_name in item.lemma_names():
                word = lemma_name.lower().replace('_', ' ')
                if len(word) <= 3:
                    continue
                
                # Check if the word contains a number using regular expressions
                if re.search(r'\d', word):
                    continue
                unique_words_expanded.add(word)
            #break
        #break
            
                
print(f'Number of unique words after WordNet expansion: {len(unique_words_expanded)}')


Number of unique words after WordNet expansion: 3014


In [111]:
unique_words_expanded

{'terminal',
 'invariable',
 'breakdown',
 'defence reaction',
 'solitaire',
 'lineage',
 'jumper cable',
 'science lab',
 'ratiocination',
 'piece of land',
 'body',
 'convenience',
 'continuance',
 'world',
 'gesture',
 'theory',
 'cadre',
 'thawing',
 'fluxion',
 'substrate',
 'equivalent',
 'adult',
 'legal philosophy',
 'connectedness',
 'uncovering',
 'writing paper',
 'decomposition',
 'input',
 'salmagundi',
 'entree',
 'cement',
 'fig tree',
 "potter's clay",
 'penny',
 'harm',
 'citrus',
 'procedure',
 'junction',
 'electric outlet',
 'deadbolt',
 'inwardness',
 'heading',
 'cranny',
 'deflection',
 'electromagnetic wave',
 'fencesitter',
 'roentgen ray',
 'strip',
 'volume',
 'rule',
 'paint',
 'neology',
 'glassful',
 "compositor's case",
 'alluviation',
 'example',
 'observation',
 'ceremony',
 'ash tree',
 'high quality',
 'exposure',
 'promontory',
 "man's clothing",
 'body substance',
 'literary genre',
 'mirror',
 'sexual activity',
 'wedlock',
 'business line',
 'grou

In [112]:
true_positive = 0 # was in the ontology and in the corpus
false_positive = 0 # was not in the ontology, but was in the corpus
true_negative = 0  # always 0
false_negative = 0 # was in the ontology, but was not in the corpus


for corpus_word in unique_words_expanded:
    if corpus_word in ontology_vocabulary:
        # was in the ontology and in the corpus
        true_positive += 1
        print(corpus_word)
    else:
        # was not in the ontology, but was in the corpus
        false_negative += 1
    
for ontology_term in ontology_vocabulary:
    if ontology_term not in unique_words_expanded:
        # was in the ontology, but was not in the corpus
        false_positive += 1
        
print(f'{true_positive=}, {false_positive=}, {true_negative=}, {false_negative=}')
precision = true_positive / (true_positive + false_positive) # fitness 
recall = true_positive / (true_positive + false_negative) # unneeded concepts

print(f'precision (relevance): {precision}, recall (coverage): {recall}, F1: {2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0}')

component
first
position
direction
kind
family
second
magnitude
lattice
basis
crystal
origin
plane
defect
true_positive=14, false_positive=39, true_negative=0, false_negative=3000
precision (relevance): 0.2641509433962264, recall (coverage): 0.0046449900464499, F1: 0.009129442451907402


In [113]:
true_positive = 0 # was in the ontology and in the corpus
false_positive = 0 # was not in the ontology, but was in the corpus
true_negative = 0  # always 0
false_negative = 0 # was in the ontology, but was not in the corpus


for corpus_word in unique_words:
    if corpus_word in ontology_vocabulary:
        # was in the ontology and in the corpus
        true_positive += 1
        print(corpus_word)
    else:
        # was not in the ontology, but was in the corpus
        false_negative += 1
    
for ontology_term in ontology_vocabulary:
    if ontology_term not in unique_words:
        # was in the ontology, but was not in the corpus
        false_positive += 1
        
print(f'{true_positive=}, {false_positive=}, {true_negative=}, {false_negative=}')
precision = true_positive / (true_positive + false_positive) # fitness 
recall = true_positive / (true_positive + false_negative) # unneeded concepts

print(f'precision (relevance): {precision}, recall (coverage): {recall}, F1: {2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0}')

magnitude
lattice
microscopy
crystal
defect
first
direction
basis
crystal_structure
component
second
plane
true_positive=12, false_positive=41, true_negative=0, false_negative=8407
precision (relevance): 0.22641509433962265, recall (coverage): 0.0014253474284356812, F1: 0.0028328611898017


In [ ]:
import os
ontology_files = os.listdir('/home/ebrahim/ontology_evaluation/ontologies/')
results = []

# Loop through each ontology file
for ontology_file in ontology_files:
    try:
        ontology_concepts_list = _owl_get_classes(ontology.sparql.graph_from(f'/home/ebrahim/ontology_evaluation/ontologies/{ontology_file}'))

        ontology_concepts = set()
        for concept, i in ontology_concepts_list:
            ontology_concepts.add(i)
        
        ontology_document = ' '.join([i.replace('_', ' ') for concept, i in ontology_concepts_list])
        print(ontology_file)
        # retrieve subject and then only related abstracts
        target_subject = ontology_document#'crystalline materials'

        # Define threshold for confidence score
        threshold = 0.95  # Example threshold, adjust as needed

        # Compute aggregated embedding for the target subject
        target_embedding = np.mean([get_word_embedding(word) for word in target_subject.split()], axis=0)

        # Compute semantic similarity and confidence score for each subject
        mapped_subjects = []
        for i, subject_embedding in enumerate(subject_embeddings):
            # Example code to compute similarity with each subject's embedding
            similarity = np.dot(target_embedding, subject_embedding) / (np.linalg.norm(target_embedding) * np.linalg.norm(subject_embedding))
            confidence_score = (similarity + 1) / 2  # Normalize to range [0, 1]
            # Filter out mapped subjects based on threshold
            if confidence_score >= threshold:
                mapped_subjects.append((list(set(all_topics))[i], confidence_score))

        # Sort mapped subjects by confidence score (highest to lowest)
        mapped_subjects = sorted(mapped_subjects, key=lambda x: x[1], reverse=True)
        target_subject_abstracts, target_subject_titles = [], []
        # Print mapped subjects with confidence scores
        for mapped_subject, confidence_score in mapped_subjects:
            #print(f"{mapped_subject}: {confidence_score}")
            target_abstracts, target_titles = papers_with_subject(document_list, titles, subjects, mapped_subject)
            target_subject_abstracts.extend(target_abstracts)
            target_subject_titles.extend(target_titles)
            
        print('len of papers in this domain', len(target_subject_titles))
        clean_text=preprocess_data(target_subject_abstracts)
        number_of_topics=100#len(clean_text)
        words=3000
        model, dictionary, doc_term_matrix =create_gensim_lsa_model(clean_text, number_of_topics,words)
        print('Done')

        unique_words = set()
        for i, _ in model.show_topics():
            topic = model.show_topic(i, topn= 10)
            for word, score in topic:
                unique_words.add(word)
                    
        print(f'number of unique words: {len(unique_words)}')
        clean_text_o = preprocess_data(ontology_concepts)
        
        model_o, dictionary_o, doc_term_matrix_o = create_gensim_lsa_model(clean_text_o, 1, words, include_bigrams=True)

        ontology_vocabulary = set()
        for i, _ in model_o.show_topics():
            topic = model_o.show_topic(i, topn=len(ontology_concepts))
            for word, score in topic:
                ontology_vocabulary.add(word)
        
        print('ontology vocab')
        print(ontology_vocabulary)
        # Initialize evaluation metrics
        true_positive = 0
        false_positive = 0
        true_negative = 0
        false_negative = 0
        print('unique_words vocab')
        print(unique_words)
        
        # Calculate True Positive and False Positive
        for corpus_word in unique_words:
            if corpus_word in ontology_vocabulary:
                true_positive += 1
                #print(corpus_word)
            else:
                false_negative += 1
        
        # Calculate False Negative
        for ontology_term in ontology_vocabulary:
            if ontology_term not in unique_words:
                false_positive += 1
        
        # Calculate Precision and Recall
        if (true_positive + false_positive) > 0:
            precision = true_positive / (true_positive + false_positive)
        else:
            precision = 0.0  # or 'undefined' or any other value you'd like to assign

        if (true_positive + false_negative) > 0:
            recall = true_positive / (true_positive + false_negative)
        else:
            recall = 0.0  # or 'undefined' or any other value you'd like to assign
        
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

        # Store the results
        results.append({
            'Ontology': ontology_file,
            'Number of concepts': len(ontology_vocabulary),
            'Precision': precision,
            'Recall': recall,
            'F1': f1,
            'Accuracy': (true_positive)  / (true_positive + false_positive + false_negative)
        })
    except:
        print('there is an error for', ontology_file)
    
# Print the table of results
print(f"{'Ontology':<30} {'Number of concepts':<30} {'Precision (Relevance)':<30} {'Recall (Coverage)':<30} {'Accuracy':<30}")
print("="*130)
for result in results:
    print(f"{result['Ontology']:<30} {result['Number of concepts']:<30} {result['Precision']:<30.4f} {result['Recall']:<30.4f} {result['Accuracy']:<30.4f}")

In [ ]:
import csv
csv_path = "coverage_recall_lsa.csv"
with open(csv_path, 'w', newline='') as csvfile:
    fieldnames = ['Ontology', 'Number of concepts', 'Precision (Relevance)', 'Recall (Coverage)', 'F1-Measure', 'Accuracy']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for result in results:
        writer.writerow({
            'Ontology': result['Ontology'],
            'Number of concepts': result['Number of concepts'],
            'Precision (Relevance)': "{:.4f}".format(result['Precision']),
            'Recall (Coverage)': "{:.4f}".format(result['Recall']),
            'F1-Measure': result['F1'],
            'Accuracy': result['Accuracy']
        })